## Data Loading
Blah

In [2]:
import sys
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score

train = pd.DataFrame.from_csv('../../train.csv')
test = pd.DataFrame.from_csv('../../test.csv')

## Dimensions of train set
ntrain,dtrain = train.shape

## Dimensions of test set
ntest, dtest = test.shape

X = train.drop(['TARGET'], axis=1)

## Add a column that tells us how many 0's we have
X['n0'] = (X == 0).sum(axis=1)

Xtest = test
Xtest['n0'] = (Xtest == 0).sum(axis=1)

## Standardize sets together
def standardize_sets(X, Xtest):
    Xtotal = X.append(Xtest)
    Xtotal_scaled = preprocessing.scale(Xtotal)
    X_scaled,Xtest_scaled = np.split(Xtotal_scaled, [ntrain])
    return [X_scaled, Xtest_scaled]

targets = np.array(train.TARGET)

## Feature Selection

In [15]:
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import f_classif,chi2
from sklearn.preprocessing import Binarizer, scale

## Top 75th percentile
p = 75
y = targets

X_bin = Binarizer().fit_transform(scale(X))
selectChi2 = SelectPercentile(chi2, percentile=p).fit(X_bin, y)
selectF_classif = SelectPercentile(f_classif, percentile=p).fit(X, y)

chi2_selected = selectChi2.get_support()
chi2_selected_features = [ f for i,f in enumerate(X.columns) if chi2_selected[i]]
f_classif_selected = selectF_classif.get_support()
f_classif_selected_features = [ f for i,f in enumerate(X.columns) if f_classif_selected[i]]
selected = chi2_selected & f_classif_selected
features = [ f for f,s in zip(X.columns, selected) if s]

## Number of features selected
print(len(features))

X_selected = X[features]
Xtest_selected = Xtest[features]


261


## Standardize data

In [16]:
X_selected,Xtest_selected = standardize_sets(X_selected, Xtest_selected)
Xs2, Xt2 = standardize_sets(X, Xtest)

## Make Validation Set

In [17]:
from sklearn import cross_validation

sss = cross_validation.StratifiedShuffleSplit(targets, 1, test_size=0.4, random_state=4224)
(train_index,valid_index) = list(sss)[0]

X_train = X_selected[train_index,]
X_valid = X_selected[valid_index,]
y_train = targets[train_index]
y_valid = targets[valid_index]

X_train2 = Xs2[train_index,]
X_valid2 = Xs2[valid_index,]

print(X_train.shape)
print(X_valid.shape)
print(X_train2.shape)
print(X_valid2.shape)

(45612, 261)
(30408, 261)
(45612, 370)
(30408, 370)


## Logistic Regression

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

logreg = LogisticRegression(penalty='l1', dual=False, max_iter=100, C=2)
logreg.fit(X_train2, y_train)

y_hat = logreg.predict_proba(X_valid2)
y_hat2 = logreg.predict_proba(X_train2)
print("Training accuracy:",logreg.score(X_train2, y_train))
print("Validation accuracy:",logreg.score(X_valid2, y_valid))
print("Training AUC:",roc_auc_score(y_train, y_hat2[:,1]))
print("Validation AUC:",roc_auc_score(y_valid, y_hat[:,1]))

('Training accuracy:', 0.96016399193194768)
('Validation accuracy:', 0.96024072612470401)
('Training AUC:', 0.8026358693607385)
('Validation AUC:', 0.79568686569827785)


In [38]:
print((logreg.coef_ != 0).sum(axis=1))
print((logreg.coef_.shape))
print(len(logreg.coef_[0]))
print([i for i in range(len(logreg.coef_[0])) if logreg.coef_[0][i] != 0])

[176]
(1, 370)
370
[0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 23, 24, 27, 28, 30, 34, 35, 39, 42, 43, 44, 45, 46, 49, 52, 54, 55, 60, 62, 63, 66, 67, 68, 69, 71, 72, 73, 74, 75, 76, 77, 78, 79, 81, 88, 89, 90, 91, 93, 94, 97, 107, 109, 110, 111, 113, 114, 117, 118, 119, 120, 121, 123, 128, 129, 135, 139, 141, 142, 145, 146, 148, 149, 150, 151, 152, 153, 154, 156, 157, 158, 164, 166, 168, 172, 178, 184, 187, 188, 190, 191, 193, 194, 195, 196, 197, 199, 200, 201, 207, 208, 209, 211, 212, 213, 219, 224, 225, 231, 235, 240, 241, 253, 255, 256, 257, 258, 259, 260, 263, 264, 270, 272, 273, 274, 276, 277, 279, 280, 281, 282, 283, 286, 289, 290, 291, 294, 295, 297, 298, 300, 301, 302, 304, 309, 310, 311, 312, 323, 324, 326, 328, 329, 330, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 348, 364, 366, 368, 369]


In [40]:
l1features = [0, 1, 2, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 23, 24, 27, 28, 30, 34, 35, 39, 42, 43, 44, 45, 46, 49, 52, 54, 55, 60, 62, 63, 66, 67, 68, 69, 71, 72, 73, 74, 75, 76, 77, 78, 79, 81, 88, 89, 90, 91, 93, 94, 97, 107, 109, 110, 111, 113, 114, 117, 118, 119, 120, 121, 123, 128, 129, 135, 139, 141, 142, 145, 146, 148, 149, 150, 151, 152, 153, 154, 156, 157, 158, 164, 166, 168, 172, 178, 184, 187, 188, 190, 191, 193, 194, 195, 196, 197, 199, 200, 201, 207, 208, 209, 211, 212, 213, 219, 224, 225, 231, 235, 240, 241, 253, 255, 256, 257, 258, 259, 260, 263, 264, 270, 272, 273, 274, 276, 277, 279, 280, 281, 282, 283, 286, 289, 290, 291, 294, 295, 297, 298, 300, 301, 302, 304, 309, 310, 311, 312, 323, 324, 326, 328, 329, 330, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 348, 364, 366, 368, 369]
X_train3 = X_train2[:,l1features]
X_valid3 = X_valid2[:,l1features]
print(X_train3.shape)
print(X_valid3.shape)

(45612, 176)
(30408, 176)


## Run the thing (Linear)

In [43]:
from sklearn import svm

lin_clf = svm.LinearSVC(dual=True, class_weight="balanced")
lin_clf.fit(X_train3, y_train)

y_hat = lin_clf.predict_proba(X_valid3)
y_hat2 = lin_clf.predict_proba(X_train3)
print("Training accuracy:",lin_clf.score(X_train3, y_train))
print("Validation accuracy:",lin_clf.score(X_valid3, y_valid))
print("Training AUC:",roc_auc_score(y_train, y_hat2))
print("Validation AUC:",roc_auc_score(y_valid, y_hat))

TypeError: __init__() got an unexpected keyword argument 'probability'

In [12]:
preds = 1./(1.+np.exp(-y_hat))
y_hatp = np.vstack((1-preds, preds)).T
print("Validation AUC:",roc_auc_score(y_valid, y_hatp[:,1]))

('Validation AUC:', 0.7153391132680198)


## Run the thing (Polynomial kernel)
This takes almost forever to run

In [44]:
from sklearn import svm
from sklearn.metrics import roc_auc_score

clfp = svm.SVC(kernel="poly", degree=3, class_weight="balanced", probability=True)
clfp.fit(X_train3, y_train)

y_hat = clfp.predict_proba(X_valid3)
y_hat2 = clfp.predict_proba(X_train3)
print("Training accuracy:",clfp.score(X_train3, y_train))
print("Validation accuracy:",clfp.score(X_valid3, y_valid))
print("Training AUC:",roc_auc_score(y_train, y_hat2))
print("Validation AUC:",roc_auc_score(y_valid, y_hat))

('Training accuracy:', 0.75291589932473912)
('Validation accuracy:', 0.74667850565640626)


ValueError: bad input shape (45612, 2)

In [45]:

print("Training AUC:",roc_auc_score(y_train, y_hat2[:,1]))

print("Validation AUC:",roc_auc_score(y_valid, y_hat[:,1]))


('Training AUC:', 0.83513594350236964)
('Validation AUC:', 0.75357108854298094)


## Run the thing (RBF kernel)
This also takes almost forever to run

In [10]:
from sklearn import svm
from sklearn.metrics import roc_auc_score

clfp = svm.SVC(kernel="rbf", class_weight="balanced")
clfp.fit(X_train, y_train)

y_hat = clfp.predict(X_valid)
y_hat2 = clfp.predict(X_train)
print("Training accuracy:",clfp.score(X_train, y_train))
print("Validation accuracy:",clfp.score(X_valid, y_valid))
print("Training AUC:",roc_auc_score(y_train, y_hat2))
print("Validation AUC:",roc_auc_score(y_valid, y_hat))

('Training accuracy:', 0.70057221783741119)
('Validation accuracy:', 0.69060773480662985)
('Training AUC:', 0.75684607177485153)
('Validation AUC:', 0.68205056331881908)
